<a href="https://colab.research.google.com/github/abriciof/IA-UFAM/blob/main/5%C2%B0_Trabalho_de_IA_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⛳ Redes Neurais Artificiais (RNA) e tradução de programa em lógica para RNA

*Turma EC01 - Engenharia da Computação*

*   Fabrício da Costa Guimarães - 21950515
*   Laura Aguiar Martinho - 21952064
*   Lorena Bastos Amazonas - 21952638
*   Mattheus Smith Costa - 21954379







## Bibliotecas

In [158]:
import math
import random
import numpy as np

## Class Neural Network

In [159]:
# Gerar random
random.seed(3)
def rand(a, b):
  return (b-a)*random.random() + a

def pesos_random(v,q):
  pesos_random = np.zeros((v+1, q))
  for i in range(v+1): #+1 bias
    for j in range(q):
      pesos_random[i][j] = rand(-2, 2)
  return pesos_random

# Função de Ativação tang_Hiper
def tang_Hiper(x, diff=False):
  if diff:
    return 1.0 - x**2
  else:
    return math.tanh(x)

class NeuralNetwork:
  def __init__(self, num_inputs, num_hidden, num_outputs, pesos):
    # input, hidden and output
    self.num_inputs = num_inputs + 1 # +1 bias
    self.num_hidden = num_hidden 
    self.num_outputs = num_outputs

    self.ai = [1.0]*self.num_inputs
    self.ah = [1.0]*self.num_hidden
    self.ao = [1.0]*self.num_outputs

    # pesos
    self.hidden_weights = np.zeros((self.num_inputs, self.num_hidden))
    self.output_weights = np.zeros((self.num_hidden, self.num_outputs))

    # pesos de entrada pré calculados
    self.hidden_weights = pesos
    # print(self.hidden_weights)

    for j in range(self.num_hidden):
      for k in range(self.num_outputs):
        self.output_weights[j][k] = rand(-0.2, 0.2)
        # self.output_weights[j][k] = w

    # ultima mudança de pesos para instante
    self.ci = np.zeros((self.num_inputs, self.num_hidden))
    self.co = np.zeros((self.num_hidden, self.num_outputs))

  # Faz previsões com a rede
  def update(self, inputs):
    if len(inputs) != self.num_inputs-1:
      raise ValueError('Número errado de entradas')

    # camada de input
    for i in range(self.num_inputs-1):
      self.ai[i] = inputs[i]

    # camada dos hidden
    for j in range(self.num_hidden):
      sum = 0.0
      for i in range(self.num_inputs):
          sum = sum + self.ai[i] * self.hidden_weights[i][j]
      self.ah[j] = tang_Hiper(sum)

    # camada de output
    for k in range(self.num_outputs):
      sum = 0.0
      for j in range(self.num_hidden):
          sum = sum + self.ah[j] * self.output_weights[j][k]
      self.ao[k] = tang_Hiper(sum)

    return self.ao[:]


  def backpropagation(self, targets, N, M):
    if len(targets) != self.num_outputs:
      raise ValueError('Número errado de entradas')

    # erros do output
    output_deltas = [0.0] * self.num_outputs
    for k in range(self.num_outputs):
      error = targets[k]-self.ao[k]
      output_deltas[k] = tang_Hiper(self.ao[k], diff=True) * error

    # erros do hidden
    hidden_deltas = [0.0] * self.num_hidden
    for j in range(self.num_hidden):
      error = 0.0
      for k in range(self.num_outputs):
          error = error + output_deltas[k]*self.output_weights[j][k]
      hidden_deltas[j] = tang_Hiper(self.ah[j], diff=True) * error

    # atualizando pesos do output
    for j in range(self.num_hidden):
      for k in range(self.num_outputs):
        change = output_deltas[k]*self.ah[j]
        self.output_weights[j][k] = self.output_weights[j][k] + N*change + M*self.co[j][k]
        self.co[j][k] = change

    # atualizando pesos do input
    for i in range(self.num_inputs):
      for j in range(self.num_hidden):
        change = hidden_deltas[j]*self.ai[i]
        self.hidden_weights[i][j] = self.hidden_weights[i][j] + N*change + M*self.ci[i][j]
        self.ci[i][j] = change

    # calculando erros
    error = 0.0
    for i in range(len(targets)):
      error = error + 0.5*(targets[i]-self.ao[i])**2

    return error


  def test(self, patterns, arredondado=False):
    if arredondado:
      for p in patterns: print(p[0], '->', [int(abs(round(a,1))) for a in self.update(p[0])])
    else:
      for p in patterns: print(p[0], '->', self.update(p[0]))


  def weights(self):
    print(' Input weights:')
    for i in range(self.num_inputs): print(self.hidden_weights[i])

    print('\n Output weights:')
    for j in range(self.num_hidden): print(self.output_weights[j])

# N: learning rate
# M: momentum factor
  def train(self, patterns, N=0.5, M=0.1):
    cont = 0
    while(True):
      cont = cont+1
      error = 0.0
      for p in patterns:
        inputs = p[0]
        targets = p[1]
        self.update(inputs)
        error = error + self.backpropagation(targets, N, M)
      if cont % 200 == 0:
        print('error %-.5f' % error)
      if error<0.001:
        print(f'\nÉpocas: {cont}\n')
        break

## CILP Translation Algorithm

### Funções principais

In [160]:
# Função de ativação semilinear bipolar (β = 1)
def h(x,b=1):
  aux = math.exp(-b*x)
  return ((2/(1 + aux)) - 1)

# Ler programa de logica
def lerP():
  fim = "n"
  P = []
  while(fim == "n"):
    cabeca = input("head: ")
    corpo = input("body: ")
    r = [cabeca, corpo.split(',')]
    P.append(r)
    # print(P)
    fim = input("end? y/n : ")
  return P

# Devolver variavel negada, ou verificar se está com negação (-D)
def neg(x, ver=False):
  
  if ver:
    if x!='':
      if x[0]=="-":
        return True
      else: return False
    else: return False

  else:
    if x[0]=="-":
      return x[1]
    else:
      return "-"+x

# Positivar todos os elementos
def negacoes(c):
  n = c[1]
  for i in range(len(n)):
    if neg(n[i], True):
      n[i]=neg(n[i])
  return n

# Lista de literais de todo programa de lógica
def literais(P):
  all = []
  for r in P:
    for s in r:
      for t in s:
        if (t not in all) and (t != '') and (neg(t) not in all):
          if neg(t,True):
            all.append(neg(t))
          else:
            all.append(t)
  return all

# Lista de literais negativos e positivos do programa de lógica
def neg_pos(P):
  Neg = []
  Pos = []
  for k in P:
    print(k)
    # for q in k:
    n=0
    p=0
    # a = P1[0][1]
    for r in k[1]:
      # print(r)
      if neg(r,True):
        print("neg: "+r)
        n+=1
      else:
        if r!='':
          print("pos:  "+r)
          p+=1

    Neg.append(n)
    Pos.append(p)
    print()
  return Neg,Pos

# q: número de clauses do programa de lógica
# v: número de literais do programa de lógica
# K: vetor com o número de literias no corpo de cada clause
def q_v_k(P):
  q = len(P)
  v = len(literais(P))
  K = []
  k = 0
  for r in P:
    # print(r[1], len(r[1]))
    if r[1]!=['']:
      K.append(len(r[1]))
    else:
      K.append(0)

  return q,v,K

# u: ocorrencia de cada head nas clauses
def u_heads(P):
  heads = [a[0] for a in P]
  return [heads.count(a) for a in heads]

# Máximo entre k e u
def Maxr(k,u):
  if k>u: return k
  else: return u

# Máximo entre o vetor k e vetor u
def Maxp(k,u):
  max = 0
  for i in k:
    for j in u:
      if Maxr(i,j)>max:
        max = Maxr(i,j)
  return max

# Cálculo do 0<Amin<1, a partir dos vetores k e u
def Amin(k,u):
  return ((Maxp(k,u)-1) / (Maxp(k,u)+1))

# Cálculo do peso W
def W_pesos(k,u,b, amin):
  return (2/b)*(math.log(1+amin)-math.log(1-amin))/(Maxp(k,u)*(amin-1) + amin+1)

# Definir a lista de limiar da camada escondida, a partir do Amin e do vetor K
def threshold_hidden(amin, K, W):
  return [(1+amin)*(k-1)*W/2 for k in K]

# Definir a lista de limiar da camada de saida, a partir do Amin e do vetor u
def threshold_output(amin, U, W):
  return [(1+amin)*(1-u)*W/2 for u in U]

# Verifica vazio
def empty(x):
  if x=='':
    return 0
  else: return 1

# Matriz de pesos para a rede neural
def pesos_clauses(P, w, amin):
  pesos = []
  for r in P:
    l = literais(P)
    l.append('Bias')
    dicio = dict.fromkeys(l, 0)
    try:
        while True:
            r[1].remove('')
    except ValueError:
        pass
    
    values = [-1 if neg(a, True) else empty(a) for a in r[1]]
    keys = negacoes(r)
    # print(keys, values)
    for i in range(len(keys)):
      dicio[keys[i]] = values[i]
    pesos.append(list(dicio.values()))
    # print(list(dicio.values()))
  
  # Multiplicando a matriz pelo peso
  for i in range(len(pesos)):
    for a in range(len(pesos[i])):
      # print(a)
      pesos[i][a]=pesos[i][a]*w

  # Deixando no shape correto para a rede neural
  pesos = np.transpose(pesos)
  # print(pesos)
  q, v, K = q_v_k(P)

  # Adicionando o limiar para camada escondida na ultima linha de pesos
  pesos[-1] = threshold_hidden(amin, K, w)
  
  return pesos

## Treino e testes

In [165]:
# Aqui voce pode ler seu programa em lógica proposicional
# P = lerP()

# Para ser mais rápido, apenas foi nomeado P no bloco de código abaixo

#### Pesos aleatórios

In [162]:
# Base para treino
input_output = [
                [[0,0,0,0,0,0], [0,1,1,0,0,0]],
                [[0,1,1,0,0,0], [0,1,0,0,0,0]],
                [[0,1,0,0,0,0], [1,1,0,0,0,0]],
                [[1,1,0,0,0,0], [1,1,0,0,0,0]]
                ]

# Programa P
P = [['A', ['B', '-C','-D']], ['A', ['E', 'F']], ['B', ['']], ['C', ['-B']]]

# Variáveis necessárias
q, v, K = q_v_k(P)
U = u_heads(P)
amin = (1 - Amin(K,U))/2 + Amin(K,U)
w = W_pesos(K, U, 1, amin)

# Pesos Aleatórios
pesos_rand = pesos_random(v,q)

# Estrutura da rede
  # v entradas
  # q camadas invisíveis
  # v saídas 
n = NeuralNetwork(v, q, v, pesos_rand)

# Treino
n.train(input_output)

# Teste
n.test(input_output, arredondado=True)

error 0.04814
error 0.04701
error 0.04026
error 0.02485
error 0.01572
error 0.00788
error 0.01459
error 0.00854
error 0.00885
error 0.00412
error 0.00439
error 0.00414
error 0.00441
error 0.00381
error 0.00290
error 0.00236
error 0.00242
error 0.00268
error 0.00272
error 0.00256
error 0.00240
error 0.00229
error 0.00211
error 0.00165
error 0.00127

Épocas: 5050

[0, 0, 0, 0, 0, 0] -> [0, 1, 1, 0, 0, 0]
[0, 1, 1, 0, 0, 0] -> [0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0] -> [1, 1, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0] -> [1, 1, 0, 0, 0, 0]


#### Pesos Gerados com CILP

In [163]:
# Base para treino
input_output = [
                [[0,0,0,0,0,0], [0,1,1,0,0,0]],
                [[0,1,1,0,0,0], [0,1,0,0,0,0]],
                [[0,1,0,0,0,0], [1,1,0,0,0,0]],
                [[1,1,0,0,0,0], [1,1,0,0,0,0]]
                ]

# Programa P
P = [['A', ['B', '-C','-D']], ['A', ['E', 'F']], ['B', ['']], ['C', ['-B']]]

# Variáveis necessárias
q, v, K = q_v_k(P)
U = u_heads(P)
amin = (1 - Amin(K,U))/2 + Amin(K,U)
w = W_pesos(K, U, 1, amin)

# Pesos gerados
pesos = pesos_clauses(P,w,amin)

# Estrutura da rede
  # v entradas
  # q camadas invisíveis
  # v saídas 
n = NeuralNetwork(v, q, v, pesos_rand)

#Treino
n.train(input_output)

#Teste
n.test(input_output, arredondado=True)

error 0.01704
error 0.01143
error 0.00653
error 0.00427
error 0.00312
error 0.00244
error 0.00200

Épocas: 1583

[0, 0, 0, 0, 0, 0] -> [0, 1, 1, 0, 0, 0]
[0, 1, 1, 0, 0, 0] -> [0, 1, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0] -> [1, 1, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0] -> [1, 1, 0, 0, 0, 0]


### Previsão

In [164]:
prev = [0,1,1,0,0,0]
prev2 = n.update(prev)
prev3 = n.update(prev2)
prev4 = n.update(prev3)
print(prev4)

[0.9967012091164039, 0.9963760787990548, -0.011486416636065772, -1.1356909259362625e-36, -1.1554265548860563e-37, -1.684502083891932e-37]
